In [ ]:
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Find out names of cols. We can't read the whole file at once, because it has troubles with structure
pre_fb = pd.read_csv('FA_data.csv', sep = ';', index_col = False, encoding='utf8', nrows = 10)
cols = pre_fb.columns

In [ ]:
# Read the whole file and change columns name
f_fb = pd.read_csv('FA_data.csv', sep = ';', index_col = False, encoding='utf8', usecols = cols)

f_fb.columns = ['Campaign name', 'Adset Name', 'Ad name', 'Creative text', 'Impressions', 'Clicks']

In [ ]:
# Encode Impressions to int and delete unsuitable entries 
for i in range(34413):
    try:
        int(f_fb['Impressions'][i])
    except:
        f_fb['Impressions'][i] = ''

In [ ]:
# Same for Clicks
for i in range(34413):
    try:
        int(f_fb['Clicks'][i])
    except:
        f_fb['Clicks'][i] = ''

In [ ]:
# Deleted entries with NA
fb = f_fb[f_fb['Impressions'] != '']

In [ ]:
# Group our entries by ad text
fb['Impressions'] = fb['Impressions'].astype(float)
fb['Clicks'] = fb['Clicks'].astype(float)

fb_g = fb.groupby('Creative text').agg({'Impressions':'sum','Clicks':'sum'})

In [ ]:
# Bring the table to a convenient view
fb_g['text'] = fb_g.index
fb_g.index = range(len(fb_g))
cols = ['text','Impressions', 'Clicks']
fb_g = fb_g[cols]

In [ ]:
# Calculate CTR
fb_g['CTR'] = ''
for i in fb_g.index:
    fb_g['CTR'][i] = fb_g['Clicks'][i] / fb_g['Impressions'][i]
fb_g

In [ ]:
# Export our data to Excel to delete all entries not in English
fb_g.to_excel('data_all_languages.xlsx')

In [ ]:
# Import our data after cleaning
fb_final = pd.read_excel('data_all_languages.xlsx', index_col = False, usecols = ['text','Impressions', 'Clicks', 'CTR'])

In [ ]:
# Describe our data
fb_final.describe()

In [ ]:
# Categorize texts with High (1), Low (0) CTR (border is 0.006) 
fb_final['CTR cat'] = ''
for i in range(546):
    if fb_final['CTR'][i] >= 0.006:
        fb_final['CTR cat'][i] = int(1)
    elif fb_final['CTR'][i] <= 0.006:
        fb_final['CTR cat'][i] = int(0)

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [ ]:
# function for text cleaning
def cleaner(text_string):
    words = re.sub('[)(,.!?#]', ' ',text_string).replace("  ", ' ')
    words2 = words.replace('  ', ' ')
    words_split = re.split(' ', words2)
    stemmer = SnowballStemmer('english')
    words_stemmed = ''
    for word in words_split:
            if re.sub("\w{1,15}", '', word) == '':
                words_stemmed += stemmer.stem(word) + ' '
            else:
                words_stemmed += ''
    words_stemmed = re.sub('\s{2,10}', ' ', words_stemmed)
    words_stemmed = re.sub('\d{1,10}', '', words_stemmed)
    
    return words_stemmed.strip()

In [ ]:
fb_clean = fb_final
fb_clean['text'] = fb_clean['text'].apply(cleaner)

In [ ]:
# Clustering our texts to more accurately divide them into training and testing samples
#!!! if one of the clusters has only 1 text, repeat clustering and only then upload it to csv
from sklearn.cluster import KMeans
km = KMeans(n_clusters=7, init='k-means++', max_iter=10000)
km.fit(X)
fb_clean['label'] = km.labels_
print(fb_clean.groupby('label').agg({'CTR':'mean', 'text':'count'}))
print(fb_clean.groupby('label').agg({'CTR':'std'}))
fb_clean.to_csv('fin_fb_clean.csv')

In [ ]:
# Make list of stop_words, which don't have semantic load and will be excluded from our analysis  
sw = ['the', 'to', 'and', 'of', 'you', 'your', 'on', 'for', 'in', 'it', 'is', 'are', 'how', 'with',
     'from', 'if', 'this', 'at', 'can', 'or', 'if', 'so', 'what', 'do', 'got', 'more', 'out', 'all',
     'our', 'one', 'them', 'that', 'be', 'thing', 'yet', 'an', 'by', 'have', 'few', 'we', 'end', 'let', 'averag',
     'has', 'not', 'recent', 'us', 'off', 'no', 'as', 'but', 'they', 'their', 'new', 'whi', 'than']

In [ ]:
# Vectorize our texts 
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True, stop_words=sw, min_df = 0.03)
cv.fit(fb_clean['text'])
X = cv.transform(fb_clean['text'])
Y = fb_clean['CTR cat']
Y = Y.astype(int)

In [ ]:
# create 5 folds for cross_val_score 
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [ ]:
# Build a logistic regression model and look at words with highest and lowest coeficients
fb_label = pd.read_csv('fin_fb_clean.csv')

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X_train, X_val, y_train, y_val = train_test_split(X, Y, train_size = 0.8, stratify = fb_label['label'], 
                                                  random_state = 1212)


lr = LogisticRegression(multi_class= 'auto', C=0.09)
lr.fit(X_train, y_train)
print('')
print ("Accuracy =", accuracy_score(y_val, lr.predict(X_val)))
print('')
print('Cross_val_score', cross_val_score(lr, X, Y, cv=kf, scoring=('accuracy')).mean())
print()
print(classification_report(y_val, lr.predict(X_val)))


 
#sanity check - let's look at the top features
#a function that fetches the feature name from vectorizer using the classifier coefficients
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), lr.coef_[0]
    )
}
#Now let's print those features from the top of the list
for best_positive in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1],
    reverse=True)[:10]:
    print('best positive', best_positive)
#And from the bottom of the list
for best_negative in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1])[:10]:
    print('best negative', best_negative)

In [ ]:
# collect words with highest and lowest coefficients for further analysis
key_words_new = []
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), lr.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1],
    reverse=True)[:10]:
    key_words_new.append(best_positive)

for best_negative in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1])[:10]:
    key_words_new.append(best_negative)
    
key_words_2 = []

for word in key_words_new: 
    key_words_2.append(str(word).split("'")[1])

key_words_2

In [ ]:
# Make a final table
for word in key_words_2:
    fb_clean[word] = ''
    fb_clean[word] = fb_clean['text'].apply(lambda text:1 if word in text else 0 )
    
d = {'word': key_words_2}
table = pd.DataFrame(d)

table['median CTR %'] = ''
for i in range(len(table)):
    table['median CTR %'][i] = round(fb_clean['CTR'][fb_clean[table['word'][i]] == 1].median() * 100, 2)
    
table['num of texts with word'] = ''
for i in range(len(table)):
    table['num of texts with word'][i] = len(fb_clean[fb_clean[table['word'][i]] == 1])
    
table

In [ ]:
# Import table to Excel for further analysis
table.to_excel('CTR_006.xlsx')

In [ ]:
###
###
###
### Now we build another model with another border of CTR
###
###
###

In [ ]:
# Categorize texts with High (1), Low (0) CTR (border is 0.01) 
fb_final['CTR cat'] = ''
for i in range(546):
    if fb_final['CTR'][i] >= 0.01:
        fb_final['CTR cat'][i] = int(1)
    elif fb_final['CTR'][i] <= 0.01:
        fb_final['CTR cat'][i] = int(0)

In [ ]:
# Apply cleaning function
fb_clean = fb_final
fb_clean['text'] = fb_clean['text'].apply(cleaner)

In [ ]:
# Vectorize our texts 
cv = CountVectorizer(binary=True, stop_words=sw)
cv.fit(fb_clean['text'])
X = cv.transform(fb_clean['text'])
Y = fb_clean['CTR cat']
Y = Y.astype(int)

In [ ]:
# Build a logistic regression model and look at words with highest and lowest coeficients
fb_label = pd.read_csv('fin_fb_clean.csv')

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X_train, X_val, y_train, y_val = train_test_split(X, Y, train_size = 0.8, stratify = fb_label['label'], 
                                                  random_state = 777)


lr = LogisticRegression(multi_class= 'auto', C=0.5)
lr.fit(X_train, y_train)
print('')
print ("Accuracy =", accuracy_score(y_val, lr.predict(X_val)))
print('')
print('Cross_val_score', cross_val_score(lr, X, Y, cv=kf, scoring=('accuracy')).mean())
print()
print(classification_report(y_val, lr.predict(X_val)))


 
#sanity check - let's look at the top features
#a function that fetches the feature name from vectorizer using the classifier coefficients
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), lr.coef_[0]
    )
}
#Now let's print those features from the top of the list
for best_positive in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1],
    reverse=True)[:10]:
    print('best positive', best_positive)
#And from the bottom of the list
for best_negative in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1])[:10]:
    print('best negative', best_negative)

In [ ]:
# collect words with highest and lowest coefficients for further analysis
key_words_new = []
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), lr.coef_[0]
    )
}

for best_positive in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1],
    reverse=True)[:10]:
    key_words_new.append(best_positive)

for best_negative in sorted(
    feature_to_coef.items(),
    key=lambda x: x[1])[:10]:
    key_words_new.append(best_negative)
    
key_words_2 = []

for word in key_words_new: 
    key_words_2.append(str(word).split("'")[1])

key_words_2

In [ ]:
# Make a final table
for word in key_words_2:
    fb_clean[word] = ''
    fb_clean[word] = fb_clean['text'].apply(lambda text:1 if word in text else 0 )
    
d = {'word': key_words_2}
table = pd.DataFrame(d)

table['median CTR %'] = ''
for i in range(len(table)):
    table['median CTR %'][i] = round(fb_clean['CTR'][fb_clean[table['word'][i]] == 1].median() * 100, 2)
    
table['num of texts with word'] = ''
for i in range(len(table)):
    table['num of texts with word'][i] = len(fb_clean[fb_clean[table['word'][i]] == 1])
    
table

In [ ]:
# Import table to Excel for further analysis
table.to_excel('CTR_01.xlsx')